# DREAM in WFM mode

This is a simulation of the DREAM chopper cascade in WFM mode.
We also show how one can convert the neutron arrival times at the detector to wavelength.

In [ ]:
import scipp as sc
import plopp as pp
import tof

## Create a source

We first create an ESS source with 2 pulses containing 500,000 neutrons each.

In [ ]:
source = tof.Source(facility='ess', neutrons=500_000, pulses=2)
source.plot()

## Component set-up

The DREAM chopper cascade consists of:

- Two counter-rotating pulse-shaping choppers (PSC) that are very close to each other, located ~6m from the source
- An overlap chopper placed right after the two PSCs
- A band control chopper
- A T0 chopper

DREAM has 5 detector banks: the Mantle, two End-caps, a High-resolution detector and a SANS detector.

For each detector bank, we use a single mean distance (in practice, one could have a different distance for each pixel).

These component parameters are pre-configured in `tof.facilities.ess`.
Here we select the configuration in high-flux mode:

In [ ]:
dream_params = tof.facilities.ess.dream(high_flux=True)
dream_params

## Run the simulation

We propagate our pulse of neutrons through the chopper cascade and inspect the results.

In [ ]:
model = tof.Model(source=source, **dream_params)
results = model.run()
results.plot(blocked_rays=5000)

## Wavelength as a function of time-of-arrival

### Plotting wavelength vs time-of-arrival

Since we know the true wavelength of our neutrons,
as well as the time at which the neutrons arrive at the detector
(coordinate named `toa` in the detector reading),
we can plot an image of the wavelengths as a function of time-of-arrival:

In [ ]:
events = sc.DataGroup()
for key, da in results.detectors.items():
    bank = da.data.flatten(to='event')
    events[key] = bank[~bank.masks['blocked_by_others']]

# Histogram and plot
events['mantle'].hist(wavelength=500, toa=500).plot(norm='log', grid=True)

### Defining a conversion from `toa` to `wavelength`

The image above shows that there is a pretty tight correlation between time-of-arrival and wavelength.

We compute the mean wavelength inside a given `toa` bin to define a relation between `toa` and `wavelength`.

In [ ]:
binned = events.bin(tof=500)

# Weighted mean of wavelength inside each bin
mu = sc.DataGroup(
    {
        key: (da.bins.data * da.bins.coords['wavelength']).bins.sum() / da.bins.sum()
        for key, da in binned.items()
    }
)

mu.plot(grid=True)

## Computing wavelengths

We set up an interpolator that will compute wavelengths given an array of `toas`.

In [ ]:
from scipp.scipy.interpolate import interp1d

wavelengths = sc.DataGroup()

for key in mu:
    # Set up interpolator
    y = mu[key].copy()
    y.coords['tof'] = sc.midpoints(y.coords['tof'])
    f = interp1d(y, 'tof', bounds_error=False)

    # Compute wavelengths
    wavs = f(events[key].coords['tof'].rename_dims(event='tof'))
    wavelengths[key] = sc.DataArray(
        data=sc.ones(sizes=wavs.sizes, unit='counts'), coords={'wavelength': wavs.data}
    ).rename_dims(tof='event')

wavelengths

We can now compare our computed wavelengths to the true wavelengths of the neutrons.

In [ ]:
pp.plot(
    {
        'wfm': wavelengths['mantle'].hist(wavelength=300),
        'original': events['mantle'].hist(wavelength=300),
    }
)